In this notebook, we will build a 3D map of a scene from a small set of images and refine it with the featuremetric optimization. We then localize an image downloaded from the Internet and show the effect of the refinement.

# Setup
We start by defining some output paths: where the intermediate files will be stored.

In [ ]:
%load_ext autoreload
%autoreload 2
import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm  # notebook-friendly progress bars
import os
import time
import sys
from hloc import extract_features, match_features, reconstruction, pairs_from_exhaustive, visualization
from hloc.visualization import plot_images, read_image
from hloc.utils.viz_3d import init_figure, plot_points, plot_reconstruction, plot_camera_colmap
from pixsfm.util.visualize import init_image, plot_points2D
from pixsfm.refine_hloc import PixSfM
from pixsfm import ostream_redirect
from PIL import Image, ImageDraw
import pycolmap
from pathlib import Path
#import visualize_model
# redirect the C++ outputs to notebook cells
cpp_out = ostream_redirect(stderr=True, stdout=True)
cpp_out.__enter__()

In [ ]:
import torch 
print(torch.__version__)
print(torch.cuda.get_arch_list())

In [ ]:
images = Path('pixsfm_dataset/')
outputs = Path('pixsfm_outputs/')
#!rm -rf $outputs
sfm_pairs = outputs / 'pairs-sfm.txt'
loc_pairs = outputs / 'pairs-loc.txt'
features = outputs / 'features.h5'
matches = outputs / 'matches.h5'
raw_dir = outputs / "raw"
ref_dir_locked = outputs / "ref_locked"

In [ ]:
sfm_input_path = Path("dense_reconstruction/output")
svo_input_path = Path("svo_output/frame_20/pointcloud/pointcloud.ply")

In [ ]:
os.listdir(sfm_input_path)

In [ ]:
sfm_model = pycolmap.Reconstruction()
sfm_model.read_binary(sfm_input_path.as_posix())

In [ ]:
sfm_model.summary()

In [ ]:
sys.path.append("/home/skumar/colmap/scripts/python")
from read_write_model import read_images_binary
sfm_images_path = sfm_input_path / "images.bin"
sfm_images_dict = read_images_binary(sfm_images_path)
for k ,v in sfm_images_dict.items(): 
    print(f"{k} => {v.name}")

In [ ]:
target_left_frame_id = 13
target_right_frame_id = 53
print(f"{sfm_images_dict[target_left_frame_id].name} {sfm_images_dict[target_right_frame_id].name}")

In [ ]:
print(type(sfm_images_dict[13]))

In [ ]:
print(sfm_images_dict[13])

In [ ]:
import numpy as np

fx = 1093.2768
fy = 1093.2768
cx = 964.989
cy = 569.276
#A = np.array([[fx,0 , cx], [0, fy, cy], [0 , 0, 1]]).astype(np.float64)

def get_camera_matrix(fx, fy, cx, cy):
    return np.array([[fx,0 , cx], [0, fy, cy], [0 , 0, 1]]).astype(np.float64)

def compute_cam_extrinsics(img):
    from read_write_model import qvec2rotmat
    R = qvec2rotmat(img.qvec)
    t = img.tvec.reshape(3,-1)
    R_t = np.concatenate((R,t), axis = 1)
    #R_t = np.vstack([np.array([0,0,0,1]), R_t])
    return R_t    #  4 * 4 matrix

def compute_projection_matrix(K, R_t):
    return np.dot(K, R_t)    


In [ ]:
left_img = sfm_images_dict[13]
left_K = get_camera_matrix(fx,fy,cx,cy)
left_Rt = compute_cam_extrinsics(left_img)
left_P = compute_projection_matrix(left_K, left_Rt)

In [ ]:
print(f"left_K: {left_K}")
print(f"left_Rt: {left_Rt}")
print(f"left_P: {left_P}")

In [ ]:
from read_write_model import read_points3D_binary
sfm_points_path = sfm_input_path / "points3D.bin"
sfm_points_dict = read_points3D_binary(sfm_points_path)

In [ ]:
len(sfm_points_dict.keys())

In [ ]:
sfm_points_keys = list(sfm_points_dict.keys())

In [ ]:
sfm_points_keys[:20]

In [ ]:
sfm_points_dict[2966350]

In [ ]:
sfm_X = np.array([value.xyz for value in sfm_points_dict.values()])

In [ ]:
sfm_X.shape

In [ ]:
ones = np.ones((sfm_X.shape[0], 1))

''' homogenezing sfm_X'''
sfm_X = np.hstack((sfm_X, ones))

In [ ]:
sfm_X.shape

In [ ]:
print(sfm_X[0])

In [ ]:
def HasPointPositiveDepth(proj_matrix, point3D):
    return np.dot(proj_matrix[2], point3D) >= np.finfo(float).eps

In [ ]:
sfm_x = np.dot(left_P, sfm_X.T).T

In [ ]:
sfm_x.shape

In [ ]:
'''
Need to filter the points with 
negative depth i.e.behind the 
camera plane before projecting
'''

sfm_x_filtered = sfm_x[sfm_x[:, 2] >= 0]

In [ ]:
sfm_x_filtered.shape

In [ ]:
sfm_x = sfm_x_filtered[:, :2] / sfm_x_filtered[:, 2:]

In [ ]:
sfm_x.shape

In [ ]:
print(sfm_x[:10,:])

In [ ]:
svo_x = svo_x[(svo_x[:, 0] < 1080) & (svo_x[:, 1] < 1920)]

In [ ]:
sfm_x_.shape

In [ ]:
sfm_images_dict[13].name

In [ ]:
from hloc.utils.viz import plot_keypoints, save_plot
from hloc.utils.io import get_keypoints
import matplotlib.pyplot as plt
import numpy as np
import os

left_image_name = sfm_images_dict[13].name
left_image_path = os.path.join(images, Path(left_image_name))

print(f"left_image_path: {left_image_path}")

plot_images([read_image(left_image_path)], dpi=50, figsize=10.5)

'''kps_list_ = [] 
for r in ref_trim_:
    kps = get_keypoints(features_path_, r)
    print(type(kps))
    kps_list_.append(kps)
'''

plot_keypoints([sfm_x[:110000]], colors = "red",  ps = .1)

current_path_ = os.getcwd()

print("current_path: ", current_path_)

print(type(current_path_))

#final_path = current_path_ + "/kps.png"


#save_plot(final_path)


In [ ]:
svo_input_path = Path("svo_output/frame_20/pointcloud/pointcloud.ply")
svo_model = pycolmap.Reconstruction()
svo_ply = svo_model.import_PLY(svo_input_path.as_posix())

In [ ]:
svo_X = np.array([point.xyz for id, point in svo_model.points3D.items()])


In [ ]:
svo_X.shape

In [ ]:
ones = np.ones((svo_X.shape[0], 1))
svo_X = np.hstack((svo_X, ones)) #homogenizing svo_X

In [ ]:
svo_x = np.dot(left_P, svo_X.T).T
svo_x_filtered = svo_x[svo_x[:, 2] >= 0]
svo_x = svo_x_filtered[:, :2] / svo_x_filtered[:, 2:]

In [ ]:
svo_x.shape

In [ ]:
svo_x_ = svo_x[svo_x[:, 1] > 1920]

In [ ]:
svo_x_.shape

In [ ]:
import cv2
test_img = cv2.imread("/home/skumar/benchmark_pixSFM/svo_output/frame_15/images/left_image.jpg")

In [ ]:
type(test_img)

In [ ]:
test_img.shape